In [ ]:
import random
import gym
import numpy as np
import tensorflow as tf
from tensorflow import keras
from collections import deque
import os
from datetime import datetime
from matplotlib import animation
import matplotlib.pyplot as plt

## Render GIF Exporter

This sinnept of code was taken from Dhruv Karthik [work](https://gist.github.com/botforge/64cbb71780e6208172bbf03cd9293553). From a list of frames we can build an animation in GIF format.

In [ ]:
def save_frames_as_gif(frames, path='./', filename='gym_animation.gif'):
    plt.figure(figsize=(frames[0].shape[1] / 72.0, frames[0].shape[0] / 72.0), dpi=72)

    patch = plt.imshow(frames[0])
    plt.axis('off')

    def animate(i):
        patch.set_data(frames[i])

    anim = animation.FuncAnimation(plt.gcf(), animate, frames = len(frames), interval=50)
    anim.save(path + filename, writer='imagemagick', fps=60)

In [ ]:
class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000)
        self.gamma = 0.95    #discount rate
        self.epsilon = 1.0  #exploration rate, starting value of epsilon, thanks to epsilon  the agen ranomly decides its action
        self.epsilon_min = 0.01 #minimum value of epsilon, so the agent can explore at least this amount
        self.epsilon_decay = 0.995 #multiplicative factor (per episode) for decreasing epsilon, so the the number of exploration decrease as the agent feels good while playing
        self.learning_rate = 0.001 #step size, how much the neural net learns in each  iteration
        self.model = self._build_model()

    def _build_model(self):
        #Neural Net for Deep-Q learning Model

        #Sequential() creates the layers.
        model = keras.models.Sequential()
        #Input lauer of state size=4 and a Hidden layer with  24 nodes
        model.add(keras.layers.Dense(24, input_dim=self.state_size, activation='relu'))
        #Hidden layer with 24 nodes
        model.add(keras.layers.Dense(24, activation='relu'))
        #Output layer with # of actions: 2 nodes, left and right
        model.add(keras.layers.Dense(self.action_size, activation='linear'))
        #Creates the model based on the previous info
        model.compile(loss='mse', optimizer=keras.optimizers.Adam(lr=self.learning_rate))
        return model

    #list of previous experiences and observations using memory
    def memorize(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    #how the agent acts
    def act(self, state):
        #at the beginning the agent select its action randomly (random between 0 and 1)
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
    
        act_values = self.model.predict(state) # predict the reward value based on the current state
        return np.argmax(act_values[0])  # returns action based on predicted reward

    #Training with experiences in the memory
    def replay(self, batch_size):
        #Samples of some experiences and call themm in minibatch
        minibatch = random.sample(self.memory, batch_size)
        states, targets_f = [], []
        #Information extracted from each memory
        for state, action, reward, next_state, done in minibatch:
            #if donde = true, target = reward
            target = reward
            if not done:
                #predict the future discounted reward, reward + whats comming in the next state
                #Q-value returns 2 outputs: one left and one right (maximum is Q-value)
                target = (reward + self.gamma * np.amax(self.model.predict(next_state)[0]))
            #with self.model tries to train itself, make the agent to approximately map the current state to future discounted reward
            target_f = self.model.predict(state)
            target_f[0][action] = target 
            # Filtering out states and targets for training
            states.append(state[0])
            targets_f.append(target_f[0])
        #the Neural Net is trained with the state and target_f    
        history = self.model.fit(np.array(states), np.array(targets_f), epochs=1, verbose=0)
        # Keeping track of loss
        loss = history.history['loss'][0]
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay
        return loss

    def load(self, name):
        self.model.load_weights(name)

    def save(self, name):
        self.model.save_weights(name)

In [ ]:
EPISODES = 1000 #number of games for the agent to play
episodes_frames = [] # list of frames for each episode
file = open(f'{datetime.now().strftime("%d-%b-%Y-%H-%M-%S")}-log.txt', 'w+') # txt file with episodes information

In [ ]:
env = gym.make('CartPole-v1') #gym environment initialized
state_size = env.observation_space.shape[0]
action_size = env.action_space.n
agent = DQNAgent(state_size, action_size)
done = False
batch_size = 32

#Iterative process
for e in range(EPISODES):
    state = env.reset() #reset state at the begining of each game
    state = np.reshape(state, [1, state_size])
    
    episodes_frames.append([]) # Adding an empty list to store episode's frames
    
    for time in range(300): #time represents each frame of the game
        episodes_frames[e].append(env.render(mode="rgb_array"))
        #decide action
        action = agent.act(state)
        #game advances to the nex frame based on the action, reward is 1  for every frame the pole survived
        next_state, reward, done, _ = env.step(action)
        reward = reward if not done else -10
        next_state = np.reshape(next_state, [1, state_size])
        #memorize previous information
        agent.memorize(state, action, reward, next_state, done)
        #next_state becomesthe current state  for next frame
        state = next_state
        
        #done = true when game ends
        if done:
            log = f'time: {datetime.now().strftime("%d-%b-%Y %H:%M:%S")}, episode: {e + 1}/{EPISODES}, score: {reward}, e: {agent.epsilon:.2}, step: {time}'
            print(log)
            file.write(log)
            break
            
        if len(agent.memory) > batch_size:
            loss = agent.replay(batch_size)
                    # Logging training loss every 10 timesteps
                if time % 10 == 0:
                    log = f'LOSS -> time: {datetime.now().strftime("%d-%b-%Y %H:%M:%S")}, episode: {e + 1}/{EPISODES}, loss: {loss:.4}, step: {time}'
                    print(log)
                    file.write(log)

file.close()
env.close()

In [ ]:
for episode, frames in enumerate(episodes_frames):
    save_frames_as_gif(frames, filename='render-episode-{episode}.gif')